In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
from tqdm import tqdm
import time
from contextlib import contextmanager
import numpy as np
from medusa.model.modeling_llama_kv import LlamaForCausalLM as KVLlamaForCausalLM
from medusa.model.medusa_model import MedusaModel, MedusaConfig
from medusa.model.kv_cache import *
from medusa.model.utils import *
from medusa.model.medusa_choices import *
import transformers
from huggingface_hub import hf_hub_download

c:\Users\admin\Downloads\Medusa-main\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\admin\Downloads\Medusa-main\.venv\Lib\site-packages\transformers\utils\generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\admin\Downloads\Medusa-main\.venv\Lib\site-packages\transformers\utils\generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
@contextmanager
def timed(wall_times, key):
    start = time.time()
    torch.cuda.synchronize()
    yield
    torch.cuda.synchronize()
    end = time.time()
    elapsed_time = end - start
    wall_times[key].append(elapsed_time)

def medusa_forward(input_ids, model, tokenizer, medusa_choices, temperature, posterior_threshold, posterior_alpha, max_steps = 512):
    wall_times = {'medusa': [], 'tree': [], 'posterior': [], 'update': [], 'init': []}
    
    with timed(wall_times, 'init'):
        if hasattr(model, "medusa_choices") and model.medusa_choices == medusa_choices:
            medusa_buffers = model.medusa_buffers
        else:
            medusa_buffers = generate_medusa_buffers(
                medusa_choices, device=model.base_model.device
            )
        model.medusa_buffers = medusa_buffers
        model.medusa_choices = medusa_choices

        if hasattr(model, "past_key_values"):
            past_key_values = model.past_key_values
            past_key_values_data = model.past_key_values_data
            current_length_data = model.current_length_data
            current_length_data.zero_()
        else:
            (
                past_key_values,
                past_key_values_data,
                current_length_data,
            ) = initialize_past_key_values(model.base_model)
            model.past_key_values = past_key_values
            model.past_key_values_data = past_key_values_data
            model.current_length_data = current_length_data

        input_len = input_ids.shape[1]
        reset_medusa_mode(model)
        medusa_logits, logits = initialize_medusa(
                input_ids, model, medusa_buffers["medusa_attn_mask"], past_key_values
        )
    new_token = 0

    for idx in range(max_steps): 
        with timed(wall_times, 'medusa'):
            candidates, tree_candidates = generate_candidates(
                    medusa_logits,
                    logits,
                    medusa_buffers["tree_indices"],
                    medusa_buffers["retrieve_indices"],
                )

        with timed(wall_times, 'tree'):
            medusa_logits, logits, outputs = tree_decoding(
                    model,
                    tree_candidates,
                    past_key_values,
                    medusa_buffers["medusa_position_ids"],
                    input_ids,
                    medusa_buffers["retrieve_indices"],
                )

        with timed(wall_times, 'posterior'):
            best_candidate, accept_length = evaluate_posterior(
                    logits, candidates, temperature, posterior_threshold, posterior_alpha
                )
        
        with timed(wall_times, 'update'):
            input_ids, logits, medusa_logits, new_token = update_inference_inputs(
                    input_ids,
                    candidates,
                    best_candidate,
                    accept_length,
                    medusa_buffers["retrieve_indices"],
                    outputs,
                    logits,
                    medusa_logits,
                    new_token,
                    past_key_values_data,
                    current_length_data,
                )

        if tokenizer.eos_token_id in input_ids[0, input_len:].tolist():
            break

    return input_ids, new_token, idx, wall_times


In [3]:
model_name = 'FasterDecoding/medusa-vicuna-7b-v1.3'

config = MedusaConfig.from_pretrained(
    model_name,
    medusa_num_heads=4,
    medusa_num_layers=1,
)

model = MedusaModel.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto"
)
tokenizer = model.get_tokenizer()

medusa_choices = mc_sim_7b_63



c:\Users\admin\Downloads\Medusa-main\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using a model of type llama to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
You are using a model of type llama to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as

In [4]:
temperature = 0.
posterior_threshold = 0.09
posterior_alpha = 0.3

prompt1

In [13]:
prompt = "Explain the concept of infinity and how it is used in calculus"


In [14]:
with torch.inference_mode():
    input_ids = tokenizer([prompt]).input_ids
    output_ids, new_token, idx, wall_time = medusa_forward(
                    torch.as_tensor(input_ids).cuda(),
                    model,
                    tokenizer,
                    medusa_choices,
                    temperature,
                    posterior_threshold,
                    posterior_alpha,
                )
    output_ids = output_ids[0][len(input_ids[0]) :]
    print("Output length:", output_ids.size(-1))
    print("Compression ratio:", new_token / idx)

Output length: 127
Compression ratio: tensor(1.0079, device='cuda:0')


In [15]:
output = tokenizer.decode(
                    output_ids,
                    spaces_between_special_tokens=False,
                )
print(output)

.
Infinity is a mathematical concept that refers to a value that is greater than any finite number. It is often represented by the symbol ∞. In calculus, infinity is used to represent the limit of a function as the input value approaches a certain value. For example, the limit of a function as x approaches infinity is infinity. This is because the function approaches a value that is greater than any finite number as the input value approaches infinity. Infinity is also used to represent the slope of a line at a point where the input value is infinity. This is known as the vertical asymptote of the function.</s>


In [16]:
max_length = 50

def format_string(text, value, max_length):
    value_str = "{:.3f}".format(value)
    return f"{text:<{max_length - len(value_str)}}{value_str}"

time_init = np.sum(wall_time['init'] )
time_medusa = np.sum(wall_time['medusa'] )
time_tree = np.sum(wall_time['tree'] )
time_posterior = np.sum(wall_time['posterior'] )
time_update = np.sum(wall_time['update'] )
time_total = time_init + time_medusa + time_tree + time_posterior + time_update

print('='*max_length)
print(format_string("Wall time init: ", time_init, max_length))
print(format_string("Wall time medusa: ", time_medusa, max_length))
print(format_string("Wall time Tree: ", time_tree, max_length))
print(format_string("Wall time Posterior: ", time_posterior, max_length))
print(format_string("Wall time Update: ", time_update, max_length))
print('-'*max_length)
print(format_string("Wall time portion medusa: ", time_medusa / time_total, max_length))
print(format_string("Wall time portion Tree: ", time_tree / time_total, max_length))
print(format_string("Wall time portion Posterior: ", time_posterior / time_total, max_length))
print(format_string("Wall time portion Update: ", time_update / time_total, max_length))
print('-'*max_length)
print(format_string("Tokens/second: ", new_token / time_total, max_length))
print('='*max_length)

Wall time init:                              0.858
Wall time medusa:                            0.114
Wall time Tree:                             77.742
Wall time Posterior:                         0.060
Wall time Update:                            0.121
--------------------------------------------------
Wall time portion medusa:                    0.001
Wall time portion Tree:                      0.985
Wall time portion Posterior:                 0.001
Wall time portion Update:                    0.002
--------------------------------------------------
Tokens/second:                               1.610


prompt2

In [17]:
prompt ="Discuss the role of geometry in architecture and how it influences building design."

In [18]:
with torch.inference_mode():
    input_ids = tokenizer([prompt]).input_ids
    output_ids, new_token, idx, wall_time = medusa_forward(
                    torch.as_tensor(input_ids).cuda(),
                    model,
                    tokenizer,
                    medusa_choices,
                    temperature,
                    posterior_threshold,
                    posterior_alpha,
                )
    output_ids = output_ids[0][len(input_ids[0]) :]
    print("Output length:", output_ids.size(-1))
    print("Compression ratio:", new_token / idx)

Output length: 512
Compression ratio: tensor(1.0020, device='cuda:0')


In [19]:
output = tokenizer.decode(
                    output_ids,
                    spaces_between_special_tokens=False,
                )
print(output)


Describe the relationship between the human body and architecture.
Explain the concept of sustainable architecture and its importance in the design of buildings.
Discuss the role of technology in architecture and how it has changed the profession.
Describe the impact of cultural and social factors on building design.
Explain the concept of adaptive reuse and its importance in preserving historic buildings.
Discuss the role of urban planning in shaping cities and communities.
Describe the impact of globalization on architecture and the profession.
Explain the concept of biophilic design and its importance in creating healthy and sustainable buildings.
Discuss the role of architectural history in shaping the profession and influencing building design.
Explain the concept of parametric architecture and its impact on building design.
Describe the impact of climate change on building design and sustainability.
Discuss the role of community engagement in the design of buildings and public s

In [20]:
max_length = 50

def format_string(text, value, max_length):
    value_str = "{:.3f}".format(value)
    return f"{text:<{max_length - len(value_str)}}{value_str}"

time_init = np.sum(wall_time['init'] )
time_medusa = np.sum(wall_time['medusa'] )
time_tree = np.sum(wall_time['tree'] )
time_posterior = np.sum(wall_time['posterior'] )
time_update = np.sum(wall_time['update'] )
time_total = time_init + time_medusa + time_tree + time_posterior + time_update

print('='*max_length)
print(format_string("Wall time init: ", time_init, max_length))
print(format_string("Wall time medusa: ", time_medusa, max_length))
print(format_string("Wall time Tree: ", time_tree, max_length))
print(format_string("Wall time Posterior: ", time_posterior, max_length))
print(format_string("Wall time Update: ", time_update, max_length))
print('-'*max_length)
print(format_string("Wall time portion medusa: ", time_medusa / time_total, max_length))
print(format_string("Wall time portion Tree: ", time_tree / time_total, max_length))
print(format_string("Wall time portion Posterior: ", time_posterior / time_total, max_length))
print(format_string("Wall time portion Update: ", time_update / time_total, max_length))
print('-'*max_length)
print(format_string("Tokens/second: ", new_token / time_total, max_length))
print('='*max_length)

Wall time init:                              0.867
Wall time medusa:                            0.151
Wall time Tree:                            315.584
Wall time Posterior:                         0.274
Wall time Update:                            0.308
--------------------------------------------------
Wall time portion medusa:                    0.000
Wall time portion Tree:                      0.995
Wall time portion Posterior:                 0.001
Wall time portion Update:                    0.001
--------------------------------------------------
Tokens/second:                               1.614


prompt-3

In [22]:
prompt = "Analyze the Fibonacci sequence and its occurrences in nature, such as in flower petals and shells."

In [23]:
with torch.inference_mode():
    input_ids = tokenizer([prompt]).input_ids
    output_ids, new_token, idx, wall_time = medusa_forward(
                    torch.as_tensor(input_ids).cuda(),
                    model,
                    tokenizer,
                    medusa_choices,
                    temperature,
                    posterior_threshold,
                    posterior_alpha,
                )
    output_ids = output_ids[0][len(input_ids[0]) :]
    print("Output length:", output_ids.size(-1))
    print("Compression ratio:", new_token / idx)

Output length: 512
Compression ratio: tensor(1.0020, device='cuda:0')


In [24]:
output = tokenizer.decode(
                    output_ids,
                    spaces_between_special_tokens=False,
                )
print(output)


Explore the golden ratio and its relationship to the Fibonacci sequence.
Investigate the Fibonacci sequence in art and architecture, such as in the design of the Parthenon.
Examine the Fibonacci sequence in finance and investing, such as in the stock market and real estate.
Discuss the Fibonacci sequence in music and rhythm, such as in the harmonics of sound waves.
Explore the Fibonacci sequence in computer science and programming, such as in algorithms and data structures.
Investigate the Fibonacci sequence in sports and games, such as in the game of chess and the movement of athletes.
Examine the Fibonacci sequence in language and linguistics, such as in the structure of words and sentences.
Discuss the Fibonacci sequence in philosophy and spirituality, such as in the concept of balance and harmony.
Explore the Fibonacci sequence in education and learning, such as in the development of problem-solving skills.
Investigate the Fibonacci sequence in technology and innovation, such as i

In [25]:
max_length = 50

def format_string(text, value, max_length):
    value_str = "{:.3f}".format(value)
    return f"{text:<{max_length - len(value_str)}}{value_str}"

time_init = np.sum(wall_time['init'] )
time_medusa = np.sum(wall_time['medusa'] )
time_tree = np.sum(wall_time['tree'] )
time_posterior = np.sum(wall_time['posterior'] )
time_update = np.sum(wall_time['update'] )
time_total = time_init + time_medusa + time_tree + time_posterior + time_update

print('='*max_length)
print(format_string("Wall time init: ", time_init, max_length))
print(format_string("Wall time medusa: ", time_medusa, max_length))
print(format_string("Wall time Tree: ", time_tree, max_length))
print(format_string("Wall time Posterior: ", time_posterior, max_length))
print(format_string("Wall time Update: ", time_update, max_length))
print('-'*max_length)
print(format_string("Wall time portion medusa: ", time_medusa / time_total, max_length))
print(format_string("Wall time portion Tree: ", time_tree / time_total, max_length))
print(format_string("Wall time portion Posterior: ", time_posterior / time_total, max_length))
print(format_string("Wall time portion Update: ", time_update / time_total, max_length))
print('-'*max_length)
print(format_string("Tokens/second: ", new_token / time_total, max_length))
print('='*max_length)

Wall time init:                              0.813
Wall time medusa:                            0.282
Wall time Tree:                            313.618
Wall time Posterior:                         0.158
Wall time Update:                            0.386
--------------------------------------------------
Wall time portion medusa:                    0.001
Wall time portion Tree:                      0.995
Wall time portion Posterior:                 0.001
Wall time portion Update:                    0.001
--------------------------------------------------
Tokens/second:                               1.624


prompt4

In [19]:
prompt = "If a car travels 60 miles in 1.5 hours, what is its speed in miles per hour?"

In [20]:
with torch.inference_mode():
    input_ids = tokenizer([prompt]).input_ids
    output_ids, new_token, idx, wall_time = medusa_forward(
                    torch.as_tensor(input_ids).cuda(),
                    model,
                    tokenizer,
                    medusa_choices,
                    temperature,
                    posterior_threshold,
                    posterior_alpha,
                )
    output_ids = output_ids[0][len(input_ids[0]) :]
    print("Output length:", output_ids.size(-1))
    print("Compression ratio:", new_token / idx)

Output length: 1
Compression ratio: tensor(inf, device='cuda:0')


In [21]:
output = tokenizer.decode(
                    output_ids,
                    spaces_between_special_tokens=False,
                )
print(output)

</s>


In [22]:
max_length = 50

def format_string(text, value, max_length):
    value_str = "{:.3f}".format(value)
    return f"{text:<{max_length - len(value_str)}}{value_str}"

time_init = np.sum(wall_time['init'] )
time_medusa = np.sum(wall_time['medusa'] )
time_tree = np.sum(wall_time['tree'] )
time_posterior = np.sum(wall_time['posterior'] )
time_update = np.sum(wall_time['update'] )
time_total = time_init + time_medusa + time_tree + time_posterior + time_update

print('='*max_length)
print(format_string("Wall time init: ", time_init, max_length))
print(format_string("Wall time medusa: ", time_medusa, max_length))
print(format_string("Wall time Tree: ", time_tree, max_length))
print(format_string("Wall time Posterior: ", time_posterior, max_length))
print(format_string("Wall time Update: ", time_update, max_length))
print('-'*max_length)
print(format_string("Wall time portion medusa: ", time_medusa / time_total, max_length))
print(format_string("Wall time portion Tree: ", time_tree / time_total, max_length))
print(format_string("Wall time portion Posterior: ", time_posterior / time_total, max_length))
print(format_string("Wall time portion Update: ", time_update / time_total, max_length))
print('-'*max_length)
print(format_string("Tokens/second: ", new_token / time_total, max_length))
print('='*max_length)

Wall time init:                              0.808
Wall time medusa:                            0.000
Wall time Tree:                              0.657
Wall time Posterior:                         0.000
Wall time Update:                            0.000
--------------------------------------------------
Wall time portion medusa:                    0.000
Wall time portion Tree:                      0.448
Wall time portion Posterior:                 0.000
Wall time portion Update:                    0.000
--------------------------------------------------
Tokens/second:                               0.683


prompt5

In [23]:
prompt="If a car travels 60 miles in 1.5 hours, what is its speed in miles per hour?"

In [24]:
with torch.inference_mode():
    input_ids = tokenizer([prompt]).input_ids
    output_ids, new_token, idx, wall_time = medusa_forward(
                    torch.as_tensor(input_ids).cuda(),
                    model,
                    tokenizer,
                    medusa_choices,
                    temperature,
                    posterior_threshold,
                    posterior_alpha,
                )
    output_ids = output_ids[0][len(input_ids[0]) :]
    print("Output length:", output_ids.size(-1))
    print("Compression ratio:", new_token / idx)


Output length: 1
Compression ratio: tensor(inf, device='cuda:0')


In [15]:
output = tokenizer.decode(
                    output_ids,
                    spaces_between_special_tokens=False,
                )
print(output)


Remote work is a type of work that is done outside of a traditional office setting. This can include working from home, a coffee shop, or any other location that is not a traditional office. On the other hand, office-based work is done in a traditional office setting, typically during regular business hours.
One of the main advantages of remote work is the flexibility it offers. With remote work, employees can set their own schedules and work from anywhere, as long as they have a reliable internet connection. This can be especially beneficial for parents who need to balance work and family responsibilities, or for people who have mobility issues that make it difficult to commute to an office.
Office-based work, on the other hand, typically requires a more structured schedule and a physical presence in the office. This can be beneficial for employees who need to collaborate with colleagues in person, or who need to be available to answer phone calls or emails during regular business ho

In [21]:
max_length = 50

def format_string(text, value, max_length):
    value_str = "{:.3f}".format(value)
    return f"{text:<{max_length - len(value_str)}}{value_str}"

time_init = np.sum(wall_time['init'] )
time_medusa = np.sum(wall_time['medusa'] )
time_tree = np.sum(wall_time['tree'] )
time_posterior = np.sum(wall_time['posterior'] )
time_update = np.sum(wall_time['update'] )
time_total = time_init + time_medusa + time_tree + time_posterior + time_update

print('='*max_length)
print(format_string("Wall time init: ", time_init, max_length))
print(format_string("Wall time medusa: ", time_medusa, max_length))
print(format_string("Wall time Tree: ", time_tree, max_length))
print(format_string("Wall time Posterior: ", time_posterior, max_length))
print(format_string("Wall time Update: ", time_update, max_length))
print('-'*max_length)
print(format_string("Wall time portion medusa: ", time_medusa / time_total, max_length))
print(format_string("Wall time portion Tree: ", time_tree / time_total, max_length))
print(format_string("Wall time portion Posterior: ", time_posterior / time_total, max_length))
print(format_string("Wall time portion Update: ", time_update / time_total, max_length))
print('-'*max_length)
print(format_string("Tokens/second: ", new_token / time_total, max_length))
print('='*max_length)

Wall time init:                              0.832
Wall time medusa:                            0.112
Wall time Tree:                            146.780
Wall time Posterior:                         0.064
Wall time Update:                            0.220
--------------------------------------------------
Wall time portion medusa:                    0.001
Wall time portion Tree:                      0.992
Wall time portion Posterior:                 0.000
Wall time portion Update:                    0.001
--------------------------------------------------
Tokens/second:                               1.588


prompt6

In [25]:

prompt = "Find the value of x  in the equation  3x + 4 = 19 ."

In [26]:
with torch.inference_mode():
    input_ids = tokenizer([prompt]).input_ids
    output_ids, new_token, idx, wall_time = medusa_forward(
                    torch.as_tensor(input_ids).cuda(),
                    model,
                    tokenizer,
                    medusa_choices,
                    temperature,
                    posterior_threshold,
                    posterior_alpha,
                )
    output_ids = output_ids[0][len(input_ids[0]) :]
    print("Output length:", output_ids.size(-1))
    print("Compression ratio:", new_token / idx)


Output length: 286
Compression ratio: tensor(1.0035, device='cuda:0')


In [27]:
output = tokenizer.decode(
                    output_ids,
                    spaces_between_special_tokens=False,
                )
print(output)



(a) 3x + 4 = 19
(b) 3x + 4 = 19
(c) 3x + 4 = 19
(d) 3x + 4 = 19

The correct answer is (d) 3x + 4 = 19.

Explanation:

The given equation is 3x + 4 = 19.

To solve for x, we need to isolate x on one side of the equation by performing the following steps:

1. Multiply both sides of the equation by the coefficient of x (3) to get a 3x on one side and 34 on the other side:
3(3x + 4) = 3(19)

1. Add 34 to both sides of the equation to get:
6x + 4 = 53

1. Divide both sides of the equation by 6 to get the final answer in the form of x:
x = 9

Therefore, the value of x in the equation 3x + 4 = 19 is 9.

Hence, the correct option is (d) 3x + 4 = 19.</s>


In [29]:
max_length = 50

def format_string(text, value, max_length):
    value_str = "{:.3f}".format(value)
    return f"{text:<{max_length - len(value_str)}}{value_str}"

time_init = np.sum(wall_time['init'] )
time_medusa = np.sum(wall_time['medusa'] )
time_tree = np.sum(wall_time['tree'] )
time_posterior = np.sum(wall_time['posterior'] )
time_update = np.sum(wall_time['update'] )
time_total = time_init + time_medusa + time_tree + time_posterior + time_update

print('='*max_length)
print(format_string("Wall time init: ", time_init, max_length))
print(format_string("Wall time medusa: ", time_medusa, max_length))
print(format_string("Wall time Tree: ", time_tree, max_length))
print(format_string("Wall time Posterior: ", time_posterior, max_length))
print(format_string("Wall time Update: ", time_update, max_length))
print('-'*max_length)
print(format_string("Wall time portion medusa: ", time_medusa / time_total, max_length))
print(format_string("Wall time portion Tree: ", time_tree / time_total, max_length))
print(format_string("Wall time portion Posterior: ", time_posterior / time_total, max_length))
print(format_string("Wall time portion Update: ", time_update / time_total, max_length))
print('-'*max_length)
print(format_string("Tokens/second: ", new_token / time_total, max_length))
print('='*max_length)

Wall time init:                              0.893
Wall time medusa:                            0.154
Wall time Tree:                            177.475
Wall time Posterior:                         0.056
Wall time Update:                            0.077
--------------------------------------------------
Wall time portion medusa:                    0.001
Wall time portion Tree:                      0.993
Wall time portion Posterior:                 0.000
Wall time portion Update:                    0.000
--------------------------------------------------
Tokens/second:                               1.601


prompt7

In [34]:
prompt = "What is the perimeter of a triangle ?"

In [35]:
with torch.inference_mode():
    input_ids = tokenizer([prompt]).input_ids
    output_ids, new_token, idx, wall_time = medusa_forward(
                    torch.as_tensor(input_ids).cuda(),
                    model,
                    tokenizer,
                    medusa_choices,
                    temperature,
                    posterior_threshold,
                    posterior_alpha,
                )
    output_ids = output_ids[0][len(input_ids[0]) :]
    print("Output length:", output_ids.size(-1))
    print("Compression ratio:", new_token / idx)


Output length: 1
Compression ratio: tensor(inf, device='cuda:0')


In [32]:
output = tokenizer.decode(
                    output_ids,
                    spaces_between_special_tokens=False,
                )
print(output)

</s>


In [33]:
max_length = 50

def format_string(text, value, max_length):
    value_str = "{:.3f}".format(value)
    return f"{text:<{max_length - len(value_str)}}{value_str}"

time_init = np.sum(wall_time['init'] )
time_medusa = np.sum(wall_time['medusa'] )
time_tree = np.sum(wall_time['tree'] )
time_posterior = np.sum(wall_time['posterior'] )
time_update = np.sum(wall_time['update'] )
time_total = time_init + time_medusa + time_tree + time_posterior + time_update

print('='*max_length)
print(format_string("Wall time init: ", time_init, max_length))
print(format_string("Wall time medusa: ", time_medusa, max_length))
print(format_string("Wall time Tree: ", time_tree, max_length))
print(format_string("Wall time Posterior: ", time_posterior, max_length))
print(format_string("Wall time Update: ", time_update, max_length))
print('-'*max_length)
print(format_string("Wall time portion medusa: ", time_medusa / time_total, max_length))
print(format_string("Wall time portion Tree: ", time_tree / time_total, max_length))
print(format_string("Wall time portion Posterior: ", time_posterior / time_total, max_length))
print(format_string("Wall time portion Update: ", time_update / time_total, max_length))
print('-'*max_length)
print(format_string("Tokens/second: ", new_token / time_total, max_length))
print('='*max_length)

Wall time init:                              0.971
Wall time medusa:                            0.000
Wall time Tree:                              0.610
Wall time Posterior:                         0.000
Wall time Update:                            0.000
--------------------------------------------------
Wall time portion medusa:                    0.000
Wall time portion Tree:                      0.386
Wall time portion Posterior:                 0.000
Wall time portion Update:                    0.000
--------------------------------------------------
Tokens/second:                               0.633
